In [287]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [288]:
df = pd.read_parquet("sell_in_norm.parquet")
prod_a_predecir = pd.read_csv('..\productos_a_predecir.txt', sep='\t')
prod_data = pd.read_parquet('prod_data.parquet')

## Pendientes
---
### operaciones sobre tn
* HECHO - razones con lags
* HECHO - marcar cero real
* HECHO - date features:
    - HECHO - mes
    - HECHO - quarter
    - HECHO - mes en el quarter
* HECHO - tiempos de monotonia de la funcion
* maximo salto
* expanding mean

### basadas en producto
* HECHO - info del producto
* porcentual de venta
    - del mes
    - del quarter
* porcentual de categoria
* HECHO - stats de categoria1 y 2
* HECHO - stats de brand

## Generales

In [289]:
# date features
df['periodo_dt'] = pd.to_datetime(df['periodo'], format='%Y%m')
df = df.sort_values(by=['product_id', 'periodo_dt'])
df['mes'] = df['periodo_dt'].dt.month
df['quarter'] = df['periodo_dt'].dt.quarter
df['month_in_quarter'] = (df['periodo_dt'].dt.month - 1) % 3 + 1
df['year'] = df['periodo_dt'].dt.year
df['yearquarter'] = df['periodo_dt'].dt.to_period('Q').astype('str').astype('category')

In [290]:
df

,product_id,periodo,tn,primer_periodo,ultimo_periodo,values,total_tn,min_tn,average_tn,median_tn,std_dev_tn,iqr_tn,max_tn,tn_norm,periodo_dt,mes,quarter,month_in_quarter,year,yearquarter
0,20001,201701,934.77222,2017-01-01,2019-12-01,36.0,50340.39558,798.01620,1398.344322,1418.02343,298.145460,335.515348,2295.19832,-1.554852,2017-01-01,1,1,1,2017,2017Q1
1,20001,201702,798.01620,2017-01-01,2019-12-01,36.0,50340.39558,798.01620,1398.344322,1418.02343,298.145460,335.515348,2295.19832,-2.013541,2017-02-01,2,1,2,2017,2017Q1
2,20001,201703,1303.35771,2017-01-01,2019-12-01,36.0,50340.39558,798.01620,1398.344322,1418.02343,298.145460,335.515348,2295.19832,-0.318592,2017-03-01,3,1,3,2017,2017Q1
3,20001,201704,1069.96130,2017-01-01,2019-12-01,36.0,50340.39558,798.01620,1398.344322,1418.02343,298.145460,335.515348,2295.19832,-1.101419,2017-04-01,4,2,1,2017,2017Q2
4,20001,201705,1502.20132,2017-01-01,2019-12-01,36.0,50340.39558,798.01620,1398.344322,1418.02343,298.145460,335.515348,2295.19832,0.348343,2017-05-01,5,2,2,2017,2017Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22344,21276,201908,0.01265,2019-03-01,2019-12-01,10.0,0.45447,0.00223,0.045447,0.02710,0.043618,0.065710,0.12249,-0.751911,2019-08-01,8,3,2,2019,2019Q3
22345,21276,201909,0.01856,2019-03-01,2019-12-01,10.0,0.45447,0.00223,0.045447,0.02710,0.043618,0.065710,0.12249,-0.616417,2019-09-01,9,3,3,2019,2019Q3
22346,21276,201910,0.02079,2019-03-01,2019-12-01,10.0,0.45447,0.00223,0.045447,0.02710,0.043618,0.065710,0.12249,-0.565291,2019-10-01,10,4,1,2019,2019Q4
22347,21276,201911,0.03341,2019-03-01,2019-12-01,10.0,0.45447,0.00223,0.045447,0.02710,0.043618,0.065710,0.12249,-0.275963,2019-11-01,11,4,2,2019,2019Q4


## Operaciones sobre tn

In [291]:
# marcar ceros
df['tn_cero'] = df['tn'] < df['median_tn'] / 10

In [292]:
# lag features
for lag in range(1, 13):
    # rolling max
    df[f'max_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).max() == df['tn_norm']
    # rolling min
    df[f'min_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).min() == df['tn_norm']
    # rolling avg
    df[f'avg_{lag}'] = df['tn_norm'].rolling(lag, min_periods=1).mean()
    # lags
    df[f'tn_lag_{lag}'] = df.groupby('product_id')['tn_norm'].shift(lag)
    # primer derivada / delta
    df[f'tn_diff_{lag}'] = df['tn_norm'].diff(lag)
    # ratios con lags
    df[f'ratio_{lag}'] = (-df[f'tn_diff_{lag}'])/df[f'tn_lag_{lag}']

for deriv in range(1, 13):
    # segunda derivada
    df[f'tn_diff2_{deriv}'] = df[f'tn_diff_{deriv}'].diff()

In [293]:
# monotonia

for i in range(13):
    df[f'crece_{i+1}'] = (df.groupby('product_id')['tn_norm'].shift(i) - df.groupby('product_id')['tn_norm'].shift(i+1)) > 0

crece_columns = df.filter(like='crece_')

df['crece_sum'] = crece_columns.sum(axis=1)
df['decrece_sum'] = 12 - crece_columns.sum(axis=1) #corregir para los primeros

## nivel producto

In [294]:
df = pd.merge(df, prod_data, on='product_id', how='left', suffixes=('', ''))

In [295]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

In [296]:
variables = ['cat1', 'cat2', 'brand']
var_stats = []

for variable in variables:
    var_stat = df.groupby(variable).apply(lambda group: pd.Series({
        f'{variable}_total': group['tn'].sum(),
        #f'{variable}_min': group['tn'].min(),
        #f'{variable}_average': group['tn'].mean(),
        f'{variable}_median': group['tn'].median(),
        #f'{variable}_std_dev': group['tn'].std(),
        #f'{variable}_iqr': (group['tn'].quantile(0.75) - group['tn'].quantile(0.25)),
        f'{variable}_max': group['tn'].max()
    }))
    var_stat.reset_index(inplace=True)
    var_stats.append(var_stat)

# Merge yearly sales with the products dataframe
#df_prod = df_prod.merge(df_temp, on='product_id', how='left')

In [297]:
for i in range(len(variables)):
    var_stat = var_stats[i]
    df = pd.merge(df, var_stat, on=variables[i], how='left')

In [298]:
tiempos = ['periodo','yearquarter','year']

for tiempo in tiempos:
    tiempo_sums = df.groupby(tiempo)['tn'].sum().reset_index()
    tiempo_sums.rename(columns={'tn': f'tn_total_{tiempo}'}, inplace=True)
    df = df.merge(tiempo_sums, on=tiempo)
    df[f'prop_product_{tiempo}'] = (df['tn'] / df[f'tn_total_{tiempo}']) * 100
    df.drop(columns=f'tn_total_{tiempo}', inplace=True)

In [299]:
variables = ['cat1','cat2','brand']

for tiempo in tiempos:
    for variable in variables:
        combi_sums = df.groupby([tiempo, variable])['tn'].sum().reset_index()
        combi_sums = df.groupby([tiempo, variable])['tn'].sum().reset_index()
        combi_sums.rename(columns={'tn': f'tn_total_{tiempo}_{variable}'}, inplace=True)
        df = df.merge(combi_sums, on=[tiempo, variable])
        df[f'prop_product_{tiempo}_{variable}'] = (df['tn'] / df[f'tn_total_{tiempo}_{variable}']) * 100
        df.drop(columns=f'tn_total_{tiempo}_{variable}', inplace=True)

## ajustes finales pre export

In [300]:
df = df.sort_values(by=['product_id', 'periodo'])

In [301]:
df.head(15)

,product_id,periodo,tn,primer_periodo,ultimo_periodo,values,total_tn,min_tn,average_tn,median_tn,...,prop_product_year,prop_product_periodo_cat1,prop_product_periodo_cat2,prop_product_periodo_brand,prop_product_yearquarter_cat1,prop_product_yearquarter_cat2,prop_product_yearquarter_brand,prop_product_year_cat1,prop_product_year_cat2,prop_product_year_brand
0,20001,201701,934.77222,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.217440,5.364205,9.941038,80.066430,1.551382,2.906793,24.486934,0.341031,0.609609,5.127212
4,20001,201702,798.01620,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.185629,4.567600,8.578260,77.736070,1.324417,2.481533,20.904526,0.291139,0.520424,4.377108
8,20001,201703,1303.35771,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.303178,5.140067,9.688766,80.287383,2.163100,4.052956,34.142258,0.475501,0.849981,7.148899
12,20001,201704,1069.96130,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.248887,4.992214,9.788178,79.904957,1.469115,2.666555,21.535487,0.390352,0.697772,5.868723
16,20001,201705,1502.20132,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.349432,5.822483,10.342820,82.754738,2.062605,3.743782,30.235334,0.548045,0.979656,8.239553
20,20001,201706,1520.06539,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.353587,5.938291,10.361742,83.792819,2.087133,3.788303,30.594891,0.554562,0.991306,8.337537
24,20001,201707,1030.67391,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.239748,4.861184,8.724250,81.058064,1.465997,2.572914,23.533215,0.376019,0.672151,5.653232
28,20001,201708,1267.39462,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.294813,5.737451,10.757726,84.132703,1.802700,3.163849,28.938221,0.462381,0.826528,6.951642
32,20001,201709,1316.94604,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.306339,4.875164,7.999196,82.221381,1.873181,3.287547,30.069621,0.480459,0.858843,7.223430
36,20001,201710,1439.75563,2017-01-01,2019-12-01,36.0,50340.39558,798.0162,1398.344322,1418.02343,...,0.334906,5.283254,8.851196,80.672567,2.036086,3.511807,28.419212,0.525263,0.938932,7.897039


In [302]:
df.to_parquet('sell_in_lag.parquet', index=False)